In [ ]:
import sys
import os

print("Current working directory:", os.getcwd())
print("Contents of /content:", os.listdir("/content"))

sys.path.insert(0, "/content/fairness-folktables")

print("Python path:")
for p in sys.path[:5]:
    print(p)


Current working directory: /content
Contents of /content: ['.config', 'fairness-folktables', '.ipynb_checkpoints', 'sample_data']
Python path:
/content/fairness-folktables
/content
/env/python
/usr/lib/python312.zip
/usr/lib/python3.12


In [ ]:
import src
print("src imported successfully")


src imported successfully


In [ ]:
from src.data import load_data
from src.fairness_utils import evaluate_model

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier


In [ ]:
from src.data import load_data
from src.fairness_utils import evaluate_model

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

# Reload CA data
X, y, sex = load_data("CA")

X_train, X_test, y_train, y_test, s_train, s_test = train_test_split(
    X, y, sex, test_size=0.3, random_state=42, stratify=y
)

# Scale for Logistic Regression
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Re-train baseline Logistic Regression
baseline_model = LogisticRegression(max_iter=2000)
baseline_model.fit(X_train_scaled, y_train)

baseline_results = evaluate_model(
    baseline_model, X_test_scaled, y_test, s_test
)

baseline_results


{'accuracy': 0.785434412265758,
 'eo': {1: np.float64(0.7644404332129964), 2: np.float64(0.6447354852016087)},
 'eo_gap': np.float64(0.11970494801138765),
 'dp': {1: np.float64(0.4675857170530122), 2: np.float64(0.30790593666594535)},
 'dp_gap': np.float64(0.15967978038706687)}

In [ ]:
gb_model.fit(X_train, y_train)

gb_results = evaluate_model(
    gb_model, X_test, y_test, s_test
)

gb_results


{'accuracy': 0.8140034071550255,
 'eo': {1: np.float64(0.7908914190502638), 2: np.float64(0.7197071259152316)},
 'eo_gap': np.float64(0.07118429313503227),
 'dp': {1: np.float64(0.46561632336798603), 2: np.float64(0.3295462742552117)},
 'dp_gap': np.float64(0.13607004911277432)}

In [ ]:
comparison = pd.DataFrame({
    "Model": ["Baseline (LR)", "Improved (GB)"],
    "Accuracy": [
        baseline_results["accuracy"],
        gb_results["accuracy"]
    ],
    "EO Gap": [
        baseline_results["eo_gap"],
        gb_results["eo_gap"]
    ],
    "DP Gap": [
        baseline_results["dp_gap"],
        gb_results["dp_gap"]
    ]
})

comparison


,Model,Accuracy,EO Gap,DP Gap
0,Baseline (LR),0.785434,0.119705,0.15968
1,Improved (GB),0.814003,0.071184,0.13607
